This code generates weather predictions for every different district in the data set (317 total districts)

In [1]:
%%time
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import siphon
import pvlib
from pvlib import solarposition, irradiance, atmosphere, pvsystem, temperature, clearsky#, inverter
from pvlib.forecast import GFS, NAM, NDFD, HRRR, RAP
from pvlib.pvsystem import PVSystem, retrieve_sam
from pvlib.modelchain import ModelChain
import seaborn
seaborn.set(style='ticks')
#pd.set_printoptions(max_columns=500)
pd.options.display.max_columns = None

/Users/josemawyin/anaconda3/lib/python3.7/site-packages/pvlib/forecast.py:19: UserWarning: The forecast module algorithms and features are highly experimental. The API may change, the functionality may be consolidated into an io module, or the module may be separated into its own package.
  'The forecast module algorithms and features are highly experimental. '


CPU times: user 889 ms, sys: 499 ms, total: 1.39 s
Wall time: 9.05 s


In [5]:
la_to_county_loc = "/Users/josemawyin/Library/Mobile Documents/com~apple~CloudDocs/Data Science Masters /698/Data/Local_Authority_District_to_County_(April_2019)_Lookup_in_England.csv"
la_to_county = pd.read_csv(la_to_county_loc)
counties_shape_centroid = pd.DataFrame(None) 
counties_shape_centroid_loc = '/Users/josemawyin/Library/Mobile Documents/com~apple~CloudDocs/Data Science Masters /698/Data/counties_shape_centroid.csv'
counties_shape_centroid = pd.read_csv(counties_shape_centroid_loc, encoding='latin-1')

###County to Region
county_to_region_loc = "/Users/josemawyin/Library/Mobile Documents/com~apple~CloudDocs/Data Science Masters /698/Data/Local_Authority_District_to_Region_(April_2019)_Lookup_in_England.csv"
county_to_region = pd.read_csv(county_to_region_loc, encoding='latin-1')
counties_shape_centroid
county_to_region=county_to_region.rename(columns = {'LAD19NM':'lad19nm'})
county_to_region = county_to_region.sort_values('lad19nm')
#=======
counties_shape_centroid = pd.merge(counties_shape_centroid, county_to_region[['RGN19NM','lad19nm']],on='lad19nm')
counties_shape_centroid.shape

(317, 13)

In [3]:
counties_shape_path = '/Users/josemawyin/Library/Mobile Documents/com~apple~CloudDocs/Data Science Masters /698/Data/Counties_LAD19NM_December_2019__Boundaries_UK_BUC-shp/Local_Authority_Districts__December_2019__Boundaries_UK_BUC.shp'
counties_shape = gpd.read_file(counties_shape_path)
counties_shape.head()

NameError: name 'gpd' is not defined

In [4]:
counties_shape_centroid_slice = counties_shape_centroid.iloc[0:5,]
#counties_shape_centroid_slice = counties_shape_centroid_slice.iloc[0:5,]
counties_shape_centroid_slice.shape
#counties_shape_centroid_slice['daily_AC_Power'] = 0

(5, 13)

In [11]:
county_lat = counties_shape_centroid_slice.columns.get_loc("lat")
county_long = counties_shape_centroid_slice.columns.get_loc("long")
#county_daily_ACP = counties_shape_centroid_slice.columns.get_loc("daily_AC_Power")
counties_shape_centroid_slice.iloc[0, county_long]

-1.2701799999999999

In [55]:
#This creates a single dataframe combining weather conditions and predicted AC Yield

#Init Variables
days_forward = 5

county_lat = counties_shape_centroid_slice.columns.get_loc("lat")
county_long = counties_shape_centroid_slice.columns.get_loc("long")
district_name = counties_shape_centroid_slice.columns.get_loc("lad19nm")
region_name = counties_shape_centroid_slice.columns.get_loc("RGN19NM")

Predictor_Response_df = pd.DataFrame(None)
n_row = len(counties_shape_centroid_slice)
#counties_shape_centroid_slice = counties_shape_centroid.iloc[0:2,]
counties_shape_centroid_slice = counties_shape_centroid
df_ACY = pd.DataFrame(index=range(0,n_row), dtype='float')
for day in range(1,2):
    n_row = len(counties_shape_centroid_slice)
    count = 0
    while count < n_row:
            
            year = 2020
            month = 11
            day = 25
            col_name = "ACY_" + str(year) + '-'+ str(month) + '-'+ str(day) 
            latitude = counties_shape_centroid_slice.iloc[count, county_lat]
            longitude = counties_shape_centroid_slice.iloc[count, county_long]
            district = counties_shape_centroid_slice.iloc[count, district_name]
            region = counties_shape_centroid_slice.iloc[count, region_name]
            tz = 'Etc/GMT'
            #year = 2011,  month = 11, day = 21, hour = 10, second = 49

            #start = pd.Timestamp(datetime.date.today(), tz=tz)
            start = pd.Timestamp(year = 2020,  month = 11, day = 25, hour = 0, second = 0, tz=tz)
            end = start + pd.Timedelta(days=days_forward)

            sandia_modules = retrieve_sam('SandiaMod')
            sapm_inverters = retrieve_sam('cecinverter')
            module = sandia_modules['Canadian_Solar_CS5P_220M___2009_']
            inverter = sapm_inverters['ABB__MICRO_0_25_I_OUTD_US_208__208V_']

            system = PVSystem(module_parameters=module,
                              inverter_parameters=inverter)

            # fx is a common abbreviation for forecast
            fx_model = GFS()
            fx_data = fx_model.get_processed_data(latitude, longitude, start, end)
            fx_data = fx_data.resample('30min').interpolate()
            fx_data['Latitude'] = latitude
            fx_data['Longitude'] = longitude
            fx_data['District'] = district
            fx_data['Region'] = region
            
            # Calculation of Clear Sky Parameters
            times = pd.date_range(start='{}-{}-{}'.format(year,month,day), end='{}-{}-{}'.format(year,month,day+days_forward), freq='30Min', tz=tz)
            solpos = pvlib.solarposition.get_solarposition(times, latitude, longitude)
            apparent_zenith = solpos['apparent_zenith']
            airmass = pvlib.atmosphere.get_relative_airmass(apparent_zenith)
            pressure = pvlib.atmosphere.alt2pres(altitude)
            airmass = pvlib.atmosphere.get_absolute_airmass(airmass, pressure)
            linke_turbidity = pvlib.clearsky.lookup_linke_turbidity(times, latitude, longitude)
            dni_extra = pvlib.irradiance.get_extra_radiation(times)
            # an input is a pandas Series, so solis is a DataFrame
            clear_df = clearsky.ineichen(apparent_zenith, airmass, linke_turbidity, altitude, dni_extra)
            clear_df.columns = ['c_ghi', 'c_dni', 'c_dhi']
            fx_data['c_ghi'] = clear_df.c_ghi
            fx_data['c_dni'] = clear_df.c_dni
            fx_data['c_dhi'] = clear_df.c_dhi
            
            
            
            

            # use a ModelChain object to calculate modeling intermediates
            mc = ModelChain(system, fx_model.location,
                            orientation_strategy='south_at_latitude_tilt')

            # extract relevant data for model chain
            mc.run_model(weather=fx_data)
            # AC Hourly Calculations
            ac = mc.ac
            daily_AC_Power = ac.sum()/220
            # Irradiance Calculations
            irradiance = mc.total_irrad
            fx_data = pd.concat([fx_data, irradiance], axis=1, sort=False)
            # Cell Temp Calculations
            cell_temp = mc.cell_temperature
            # Adding Columns to DataFrame
            fx_data['Cell_temp'] = cell_temp
            fx_data['hourly_AC_Power'] = ac/220
            fx_data['daily_AC_Power'] = daily_AC_Power
            fx_data.index.name = 'TIMESTAMP'
            fx_data.reset_index(inplace=True)
            fx_data['Daily_Hour'] = fx_data.TIMESTAMP.dt.hour + fx_data.TIMESTAMP.dt.minute / 60
            fx_data['Day_of_Year'] = fx_data['TIMESTAMP'].dt.dayofyear
            fx_data['DateTime'] = pd.to_datetime(fx_data['TIMESTAMP']).dt.strftime('%d%b%Y:%H:%M')
            #fx_data['DateTime'] = pd.to_datetime(fx_data['TIMESTAMP'], format='%d%b%Y:%H:%M')
            
            
            Predictor_Response_df = pd.concat([Predictor_Response_df,fx_data])
            
            df_ACY.loc[count, col_name] = daily_AC_Power
            Predictor_Response_df.to_csv('/Users/josemawyin/Library/Mobile Documents/com~apple~CloudDocs/Data Science Masters /698/Data/Predictor_Response_11-25to11-30.csv')
            count += 1
Predictor_Response_df.to_csv('/Users/josemawyin/Library/Mobile Documents/com~apple~CloudDocs/Data Science Masters /698/Data/Predictor_Response_11-25to11-30.csv')
Predictor_Response_df    

,TIMESTAMP,temp_air,wind_speed,ghi,dni,dhi,total_clouds,low_clouds,mid_clouds,high_clouds,Latitude,Longitude,District,Region,c_ghi,c_dni,c_dhi,poa_global,poa_direct,poa_diffuse,poa_sky_diffuse,poa_ground_diffuse,Cell_temp,hourly_AC_Power,daily_AC_Power,Daily_Hour,Day_of_Year,DateTime
0,2020-11-25 00:00:00+00:00,8.090302,6.179259,0.0,0.0,0.0,58.000000,52.000000,0.000000,15.000000,54.676140,-1.27018,Hartlepool,North East,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,8.090302,-0.000341,4.560344,0.0,330,25Nov2020:00:00
1,2020-11-25 00:30:00+00:00,8.250249,6.128551,0.0,0.0,0.0,64.166664,59.166668,3.333333,19.166666,54.676140,-1.27018,Hartlepool,North East,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,8.250249,-0.000341,4.560344,0.5,330,25Nov2020:00:30
2,2020-11-25 01:00:00+00:00,8.410197,6.077842,0.0,0.0,0.0,70.333336,66.333336,6.666667,23.333334,54.676140,-1.27018,Hartlepool,North East,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,8.410197,-0.000341,4.560344,1.0,330,25Nov2020:01:00
3,2020-11-25 01:30:00+00:00,8.570145,6.027134,0.0,0.0,0.0,76.500000,73.500000,10.000000,27.500000,54.676140,-1.27018,Hartlepool,North East,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,8.570145,-0.000341,4.560344,1.5,330,25Nov2020:01:30
4,2020-11-25 02:00:00+00:00,8.730092,5.976426,0.0,0.0,0.0,82.666664,80.666664,13.333333,31.666666,54.676140,-1.27018,Hartlepool,North East,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,8.730092,-0.000341,4.560344,2.0,330,25Nov2020:02:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,2020-11-29 22:00:00+00:00,6.451192,1.756037,0.0,0.0,0.0,0.333333,0.333333,0.000000,0.000000,51.512199,-0.15295,Westminster,London,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,6.451192,-0.000341,11.013994,22.0,334,29Nov2020:22:00
237,2020-11-29 22:30:00+00:00,6.375900,2.196925,0.0,0.0,0.0,0.500000,0.500000,0.000000,0.000000,51.512199,-0.15295,Westminster,London,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,6.375900,-0.000341,11.013994,22.5,334,29Nov2020:22:30
238,2020-11-29 23:00:00+00:00,6.300608,2.637813,0.0,0.0,0.0,0.666667,0.666667,0.000000,0.000000,51.512199,-0.15295,Westminster,London,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,6.300608,-0.000341,11.013994,23.0,334,29Nov2020:23:00
239,2020-11-29 23:30:00+00:00,6.225317,3.078701,0.0,0.0,0.0,0.833333,0.833333,0.000000,0.000000,51.512199,-0.15295,Westminster,London,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,6.225317,-0.000341,11.013994,23.5,334,29Nov2020:23:30
